CISC 7700X HW# 5: Using data from: spambase, build a Naive Bayes email classifier. Nothing too fancy, just a training module, and a classifier module. Submit code and accuracy you get on the spambase dataset. [hint]

In [1]:
#http://theparticle.com/cs/bc/dsci/20211107bayes.tar.gz
!pip install requests

import requests
import tarfile
import io

def extract_tar_file(url, extract_path):
    # Step 1: Stream download the file
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Check for HTTP errors

    # Create a BytesIO buffer to hold the tar.gz data
    tar_buffer = io.BytesIO()

    # Write the streamed content to the buffer
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            tar_buffer.write(chunk)

    # Move the buffer's cursor to the beginning
    tar_buffer.seek(0)

    # Step 2: Extract the tar.gz file from the buffer
    with tarfile.open(fileobj=tar_buffer, mode='r:gz') as tar:
        tar.extractall(path=extract_path)
        print(f"Extracted contents to {extract_path}")

# Example Usage
if __name__ == "__main__":
    url = "http://theparticle.com/cs/bc/dsci/20211107bayes.tar.gz"  # Replace with your URL
    extract_path = "/home/studio-lab-user/sagemaker-studiolab-notebooks/Cisc7700/Cisc7700_hw5" # Directory to extract the contents !pwd

    extract_tar_file(url, extract_path)


Extracted contents to /home/studio-lab-user/sagemaker-studiolab-notebooks/Cisc7700/Cisc7700_hw5


In [ ]:
import os
import math

# Initialize global dictionaries and counters
catcnttot = 0  # Total number of categories (documents)
catcnt = {}    # Count of each category
model = {}     # Word counts per category
modeltot = {}  # Total word counts per category

def train(cats):
    global catcnttot
    
    for cat in cats:
        for filename in os.listdir(cat):
            if filename.endswith(".txt"):
                file_path = os.path.join(cat, filename)
    
                if cat in catcnt:
                    catcnt[cat] += 1
                else:
                    catcnt[cat] = 1

                catcnttot += 1

                with open(file_path, 'r', encoding='utf-8') as infile:
                    content = infile.read()
                    words = content.lower().split()

                    for word in words:
                        if cat not in model:
                            model[cat] = {}
                        
                        if word in model[cat]:
                            model[cat][word] += 1
                        else:
                            model[cat][word] = 1

                        if cat in modeltot:
                            modeltot[cat] += 1
                        else:
                            modeltot[cat] = 1
    
    for filename in os.listdir("test"):
        if filename.endswith(".txt"):
            file_path = os.path.join("test", filename)
            with open(file_path, 'r', encoding='utf-8') as infile:
                content = infile.read()
                words = content.lower().split()
            result = {}
            for cat in cats:
                if cat in catcnt:
                    result[cat] = math.log(catcnt[cat] / catcnttot)
                else:
                    result[cat] = math.log(1 / catcnttot)

                for word in words:
                    if cat in model and word in model[cat]:
                        result[cat] += math.log(model[cat][word] / modeltot[cat])
                    else:
                        result[cat] += math.log(0.001)

            for cat in result:
                result[cat] = math.exp(result[cat])

            total = 0
            for it in result.values():
                total += it
            for cat in result:
                result[cat] = result[cat] / total

            print(f"{filename}: spam={result['spam']}; ham={result['ham']}")

folder=["spam", "ham"]
train(folder)

